## __0. Bibliotecas__

In [ ]:
# mover os imports para cá

## __1. Datasets__

In [ ]:
# carregar todos os datasets aqui

## __2. Pré-processamento dos dados__

Uma vez que vamos trabalhar apenas com pontos no plano cartesiano, precisamos transformar nossos dados em uma **matriz bidimensional**. Para isso, utilizaremos a técnica **SVD** (Singular Value Decomposition) que reduz uma matriz *n*-dimensional em uma matriz *k*-dimensional, onde *k* é o número de fatores latentes. Para nosso trabalho, *k* é sempre 2.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
iris = datasets.load_iris() # apenas o iris tem duas classes linearmente separáveis
digits = datasets.load_digits()
wine = datasets.load_wine()
cancer = datasets.load_breast_cancer()

df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['Target'] = pd.DataFrame(iris.target)


In [3]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=2, n_iter=4, random_state=42)
data_points = svd.fit_transform(df)

In [ ]:
df_new = pd.DataFrame(data_points, columns=['x','y'])
df_new['Target'] = pd.DataFrame(iris.target)
df_new

## __3. Envoltória Convexa__
 

#### Implementação do algoritmo de Gift Wrapping(Jarvis March) para determinar as envoltórias das amostras. Este algoritmo possui complexidade $O(nh)$, sendo $n$ o número de pontos e $h$ os pontos que efetivamente estão na envoltória.

In [5]:
# classe para represemtar segmentos de reta no espaço
class Segment:
    def __init__(self, left, right):
        self.left = left
        self.right = right

In [6]:
# classe para representar pontos bidimensionais no espaço
class Ponto:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [7]:
#calculando o produto vetorial entre segmentos ab e ac, a = ancora
def produto_vetorial(ancora, b, c):
    return ((b.x-ancora.x)*(c.y-ancora.y) - (c.x-ancora.x)*(b.y-ancora.y))


In [8]:
# encontrando o ponto mais embaixo em relação a y
def mais_a_esquerda(pontos):
    min = 0
    for i in range(1,len(pontos)):
        if pontos[i].y < pontos[min].y:
            min = i
        elif pontos[i].y == pontos[min].y:
            if pontos[i].x < pontos[min].x: # em caso de empate, escolhemos o ponto mais a esquerda
                min = i
    return min

In [9]:
# funcao que recebe um vetor de pontos, e retorna lista com pontos da envoltoria convexa
def gift_wrapping(pontos):
    i_esquerda = mais_a_esquerda(pontos) # guarda indice do ponto mais a esquerda
    p = pontos[i_esquerda]

    prox = (i_esquerda + 1) % len(pontos)

    teste = 0

    envoltoria = [] #incializa envoltoria vazia
    envoltoria.append(p) # adiciona ponto mais à esquerda a envoltoria

    q = p
    while(True):
        # calcula o prox ponto da envoltoria
        while(teste != len(pontos)):
            det = produto_vetorial(q, pontos[prox], pontos[teste])
            if det < 0: # se for negativo, pontos[teste] tem o menor angulo polar,sendo o prox ponto da envoltoria
                prox = teste
            # atualiza teste
            teste = teste+1;

        # se tivermos retornados a ancora, loop acaba
        if p == pontos[prox]:
            break

        # adiciona ponto a envoltoria
        envoltoria.append(pontos[prox])

        # atualiza variáveis
        q = pontos[prox]
        prox = (prox + 1) % len(pontos)
        teste = 0

    return envoltoria

In [10]:
pontos = [Ponto(x, y) for x, y in data_points]
envoltoria = gift_wrapping(pontos)

In [ ]:
# Criando listas separadas para nomes e idades
Xs1 = [ponto.x for ponto in pontos]
Ys1 = [ponto.y for ponto in pontos]

Xs2 = [ponto.x for ponto in envoltoria]
Ys2 = [ponto.y for ponto in envoltoria]

# Adicionando o primeiro ponto da envoltória ao final para fechar a forma
Xs2.append(envoltoria[0].x)
Ys2.append(envoltoria[0].y)

# Plotando um gráfico de barras
plt.scatter(Xs1, Ys1, c=iris.target, s=20)
plt.plot(Xs2, Ys2, linestyle='-', marker='o', color='orange')
plt.xlabel('x')
plt.ylabel('x')
plt.title('Envoltória Convexa')
plt.show()

In [ ]:
# testando envoltoria com a matriz bidimensional
plt.clf()
plt.figure(figsize = (10, 6))
names = iris.target_names
label = (iris.target).astype(int)
colors = ['b','r','g']
for i in range(len(names)):
    bucket = df_new[df_new['Target'] == i]
    bucket = bucket.iloc[:,[0,1]].values
    bucket_points = [Ponto(x, y) for x, y in bucket]
    envoltoria[i] = gift_wrapping(bucket_points)
    Xs1 = [ponto.x for ponto in bucket_points]
    Ys1 = [ponto.y for ponto in bucket_points]
    plt.scatter(Xs1, Ys1, s=20)
    Xs2 = [ponto.x for ponto in envoltoria[i]]
    Ys2 = [ponto.y for ponto in envoltoria[i]]
    Xs2.append(envoltoria[i][0].x)
    Ys2.append(envoltoria[i][0].y)
    for j in envoltoria:
        plt.plot(Xs2, Ys2, linestyle='-', marker='o', color='orange')
plt.legend()
plt.show()

In [ ]:
#testando envoltoria com matriz inteira
plt.clf()
plt.figure(figsize = (10, 6))
names = iris.target_names
label = (iris.target).astype(int)
colors = ['b','r','g']
plt.title('Petal Width vs Petal Length')
plt.xlabel(iris.feature_names[2])
plt.ylabel(iris.feature_names[3])
for i in range(len(names)):
    bucket = df[df['Target'] == i]
    bucket = bucket.iloc[:,[2,3]].values
    bucket_points = [Ponto(x, y) for x, y in bucket]
    envoltoria[i] = gift_wrapping(bucket_points)
    Xs1 = [ponto.x for ponto in bucket_points]
    Ys1 = [ponto.y for ponto in bucket_points]
    plt.scatter(Xs1, Ys1, s=20)
    Xs2 = [ponto.x for ponto in envoltoria[i]]
    Ys2 = [ponto.y for ponto in envoltoria[i]]
    Xs2.append(envoltoria[i][0].x)
    Ys2.append(envoltoria[i][0].y)
    for j in envoltoria:
        plt.plot(Xs2, Ys2, linestyle='-', marker='o', color='orange')
plt.legend()
plt.show()

## __4. Varredura Linear__ 


#### Implementação da varredura linear para verificação de intercessão de segmentos das envoltórias. 


In [ ]:
def on_segment(p1, p2, p):
    return min(p1.x, p2.x) <= p.x <= max(p1.x, p2.x) and min(p1.y, p2.y) <= p.y <= max(p1.y, p2.y)

In [ ]:
def intersect(p1, p2, p3, p4):
    d1 = produto_vetorial(p3, p4, p1)
    d2 = produto_vetorial(p3, p4, p2)
    d3 = produto_vetorial(p1, p2, p3)
    d4 = produto_vetorial(p1, p2, p4)

    if ((d1 > 0 and d2 < 0) or (d1 < 0 and d2 > 0)) and \
        ((d3 > 0 and d4 < 0) or (d3 < 0 and d4 > 0)):
        return True

    elif d1 == 0 and on_segment(p3, p4, p1):
        return True
    elif d2 == 0 and on_segment(p3, p4, p2):
        return True
    elif d3 == 0 and on_segment(p1, p2, p3):
        return True
    elif d4 == 0 and on_segment(p1, p2, p4):
        return True
    else:
        return False

In [ ]:
def any_segment_intersect(envoltoria_1, envoltoria_2):
    n = len(envoltoria_1)
    m = len(envoltoria_2)
    for i in range(n):
        for j in range(m):
            if intersect(envoltoria_1[i], envoltoria_1[(i + 1) % n], envoltoria_2[j], envoltoria_2[(j + 1) % m]):
                return True
                
    return False

In [ ]:
res1 = any_segment_intersect(envoltoria[0], envoltoria[1]) # deve retornar False
res2 = any_segment_intersect(envoltoria[1], envoltoria[2]) # deve retornar True
res3 = any_segment_intersect(envoltoria[0], envoltoria[2]) # deve retornar False

In [ ]:
print(res1)
print(res2)
print(res3)

## __5. Algoritmo de varredura para achar pontos mais próximos entre dois conjuntos__


##### __5.1) Implementando a classe PontoRotulado:__


In [ ]:
#classe ponto rotulado, contém um Ponto e um rótulo(0 ou 1) (necessária para rodar o closest pair)
class PontoRotulado:
    def __init__(self, x, y, rotulo):
        self.ponto = Ponto(x, y)
        self.rotulo = rotulo

#### __5.2) Algoritmo de varredura__ 
Este algoritmo tem em seu melhor caso complexiadade $O(n.log(n))$. Em seu pior caso, essa complexidade aumenta para $O(n^2)$. Dada a restrição de que cada ponto do par de pontos retornados precisa ser de um conjunto de pontos diferentes, o parâmetro de distância mínima, usado no algoritmo, tende a ter valores maiores do que se executássemos o algoritmo sem essa restrição. Portanto, espera-se que a complexidade executada esteja entre o caso médio e o pior caso do algoritmo.

In [1]:
import sys
import math
 
# função que retorna par de pontos rotulados mais próximo entre dois conjuntos de pontos rotulados
def par_mais_proximo(pontos):
    # inicializando variáveis
    p1 = (0,0,0)
    p2 = p1

    # ordena a lista de objetos com base nas coordenadas x dos pontos
    ordenados_X = sorted(pontos, key=lambda obj: obj.ponto.x)

    # menor distancia inicializada com valor alto
    min = sys.maxsize
 
    # criando objeto do tipo conjunto
    st = set()
    # adiciona ponto mais a esquerda ao conjunto
    st.add(ordenados_X[0])
 
    # varredura sobre pontos ordenados em X
    for i in range(1, len(ordenados_X)):
        # cria conjunto de pontos a esquerda do ponto atual numa distancia no máximo d
        l = set([p for p in st if (p.ponto.x >= ordenados_X[i].ponto.x - min) and (p.ponto.y >= ordenados_X[i].ponto.y - min) ])
        # conjunto de pontos a direita do ponto atual a uma distancia no máximo d
        r = set([p for p in st if (p.ponto.x <= ordenados_X[i].ponto.x + min) and (p.ponto.y <= ordenados_X[i].ponto.y + min)])
        # intercessao de pontos que podem ter distancia menor que d entre si
        intersection = l & r
        
        # se não houver pontos com distancias possivelmente menores, passa para o proximo ponto ordenado
        if len(intersection) == 0:
            continue

        # loop sobre a interseção dos conjuntos l e r
        for val in intersection:
            if ordenados_X[i].rotulo == val.rotulo:
                continue

            # Calcula a distância entre os pontos
            dist = math.sqrt(math.pow(ordenados_X[i].ponto.x - val.ponto.x, 2) + math.pow(ordenados_X[i].ponto.y - val.ponto.y, 2))

            # Atualiza a distância mínima, se necessário
            if min > dist:
                min = dist
                # guardando o par de pontos mais prox até o momento
                p1 = ordenados_X[i]
                p2 = val

        st.add(ordenados_X[i])
 
    return p1, p2
 

## __6. Equação da reta classificadora__


##### Para qualquer equação linear $y = mx + b$, as perpendiculares à ela terão todas uma inclinação de $-1/m$, o recíproco oposto da inclinação original. Para encontrar uma reta perpendicular de um dado segmento $p1p2$, sendo que esta reta também passa por um ponto específico $(x, y)$, nesse caso, a mediana do segmento, basta resolver a equação $y = (-1/m)x + b$, substituindo os valores conhecidos de $m$, $x$ e $y$ para resolver $b$.

In [2]:
# retorna  m e b da equação da reta perpendicular
def reta_perpendicular(p1, p2):
    inclinacao_original = (p1.y-p2.y)/(p1.x-p2.x)
    
    inclinacao_perpendicular = (-1.0)/inclinacao_original
    # calcula a mediana
    mediana = ((p1.x + p2.x)/2, (p1.y + p2.y)/2) # ponto da mediana
    print("mediana %d", mediana)
    
    b = mediana[1] - inclinacao_perpendicular * mediana[0]
    return (b, inclinacao_perpendicular)

# função para calcular os valores y da reta dada um inclinação m e intercessao b
def reta(m, b, x):
    return m * x + b